In [1]:
# Importing the required libraries
import cv2
import time
import numpy as np
import os, shutil
import math
import sklearn

In [2]:
# Defining a method which returns trained YOLO models with different weights
def loadYolo(yolowgt):
    if yolowgt == 320:
        print('High speed, Less accuracy')
        net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    elif yolowgt == 416:
        print('Moderate speed, Moderate accuracy')
        net = cv2.dnn.readNet("yolov3-416.weights", "yolov3.cfg")
    elif yolowgt == 608:
        print('Less speed, High accuracy')
        net = cv2.dnn.readNet("yolov3-608.weights", "yolov3.cfg")
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layers_names = net.getLayerNames()
    output_layers = [layers_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
    return net, classes, colors, output_layers

# Method to calculate the distance between 2 given points
def getDistance(x1,x2,y1,y2):
    return math.sqrt((x1-x2)**2 + (y1-y2)**2)

# To create the detection box
def getDetectionBox(frame,x1,y1,x2,y2,color):
    cv2.rectangle(frame,(x1,y1),(x2,y2), color, 2)
    xmid = ((x1+x2)/2)
    ymid = ((y1+y2)/2)
    cv2.circle(frame, (int(xmid),int(ymid)), 3, ORANGE, 2)

# Defining colors which we used for bounding box, centroids.
GREEN = (0,255,0)
RED = (0,0,255)
YELLOW = (0,255,255)
ORANGE = (0,165,255)
GREY = (192,192,192)
BLACK = (0,0,0)

#Min distance for social distance violation
MIN_DISTANCE = 60

In [3]:
#Method to detect social distance detection
#Ref: https://github.com/deepak112/Social-Distancing-AI
#Ref: https://github.com/nandinib1999/object-detection-yolo-opencv/blob/master/yolo.py

def social_distance_detection(mode):
    cap = cv2.VideoCapture('data/demo.mp4')
    count = 0
    global image
    while True:
        highCount, lowCount= 0, 0
        centroids = []
        box_colors = []
        boxesDetected = []
        ret, image_frame = cap.read() 
        if ret:
            # resize image_frame for prediction
            dim = (416, 416)
            image_frame_resized = cv2.resize(image_frame, dim)
            
        else:
            break
        
        # Bird eye view transformation - Error in code, so commented it!
        '''
        (H, W) = image_frame_resized.shape[:2]
        
        if count == 0:
            while True:
                image = image_frame_resized
                cv2.imshow("image", image_frame_resized)
                cv2.waitKey(1)
                if len(mouse_pts) == 8:
                    cv2.destroyWindow("image")
                    break
               
            points = mouse_pts      
        src = np.float32(np.array(points[:4]))
        dst = np.float32([[0, H], [W, H], [W, 0], [0, 0]])
        prespective_transform = cv2.getPerspectiveTransform(src, dst)

        # using next 3 points for horizontal and vertical unit length(in this case 180 cm)
        pts = np.float32(np.array([points[4:7]]))
        warped_pt = cv2.perspectiveTransform(pts, prespective_transform)[0]
        
        distance_w = np.sqrt((warped_pt[0][0] - warped_pt[1][0]) ** 2 + (warped_pt[0][1] - warped_pt[1][1]) ** 2)
        distance_h = np.sqrt((warped_pt[0][0] - warped_pt[2][0]) ** 2 + (warped_pt[0][1] - warped_pt[2][1]) ** 2)
        pnts = np.array(points[:4], np.int32)
        cv2.polylines(image_frame_resized, [pnts], True, (70, 70, 70), thickness=2)
        '''
        image_frame_rgb = cv2.cvtColor(image_frame, cv2.IMREAD_COLOR)
        height, width, channels = image_frame.shape

        cols = image_frame_resized.shape[1] 
        rows = image_frame_resized.shape[0]
    
        # Detecting objects
        blob = cv2.dnn.blobFromImage(image_frame_resized, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
        model.setInput(blob)
        outs = model.forward(output_layers)

        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                #if prediction is 60% and class id is 0 (person - can check in the coco.names file)
                if confidence > 0.6 and class_id == 0:

                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
        
        # Apply non-max suppression (a technique used in numerous computer vision tasks. 
        # It is a class of algorithms to select one entity (e.g., bounding boxes) out of many overlapping entities.)
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) #conf_threshold, nms_threshold
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                xmin = x
                ymin = y
                xmax = (x + w)
                ymax = (y + h)
                
                #Centroid
                xmid = ((xmax+xmin)/2)
                ymid = ((ymax+ymin)/2)
                centroid = (xmid,ymid)
                
                # If we know the focal length then we can use the following snippet to convert mid points to cm
                '''
                distance = (165 * F)/h
                xmid_cm = (xmid * distance) / F
                ymid_cm = (ymid * distance) / F
                '''
                boxesDetected.append([xmin,ymin,xmax,ymax,centroid])

                my_color = 0
                for k in range (len(centroids)): #This loop won't run first time as the list is empty
                    previousCentroid = centroids[k]
                    #Pass co-ordinates of current box and previous detected box and get the distance
                    if getDistance(previousCentroid[0],centroid[0],previousCentroid[1],centroid[1]) < MIN_DISTANCE:
                        box_colors[k] = 1
                        my_color = 1
                        cv2.line(image_frame, (int(previousCentroid[0]),int(previousCentroid[1])), (int(centroid[0]),int(centroid[1])), YELLOW, 1,cv2.LINE_AA)
                        cv2.circle(image_frame, (int(previousCentroid[0]),int(previousCentroid[1])), 3, ORANGE, -1,cv2.LINE_AA)
                        cv2.circle(image_frame, (int(centroid[0]),int(centroid[1])), 3, ORANGE, -1,cv2.LINE_AA)
                        break
                centroids.append(centroid)
                box_colors.append(my_color)
        
        for i in range (len(boxesDetected)):
            x0 = boxesDetected[i][0]
            y0 = boxesDetected[i][1]
            x1 = boxesDetected[i][2]
            y1 = boxesDetected[i][3]
            # Bounding box of green color
            if box_colors[i] == 0:
                getDetectionBox(image_frame,x0,y0,x1,y1,GREEN)
                labelSize, baseLine = cv2.getTextSize("SAFE", cv2.FONT_HERSHEY_DUPLEX, 0.5, 2)

                y0label = max(y0, labelSize[1])
                cv2.rectangle(image_frame, (x0, y0label - labelSize[1]),(x0 + labelSize[0], y0 + baseLine), GREEN, cv2.FILLED)
                cv2.putText(image_frame, "SAFE", (x0, y0), cv2.FONT_HERSHEY_DUPLEX, 0.3, BLACK, 1)
                lowCount += 1
            #Bounding box of red color
            else:
                getDetectionBox(image_frame,x0,y0,x1,y1,RED)
                labelSize, baseLine = cv2.getTextSize("UNSAFE", cv2.FONT_HERSHEY_DUPLEX, 0.5, 2)

                y0label = max(y0, labelSize[1])
                cv2.rectangle(image_frame, (x0, y0label - labelSize[1]),(x0 + labelSize[0], y0 + baseLine), RED, cv2.FILLED)
                cv2.putText(image_frame, "UNSAFE", (x0, y0), cv2.FONT_HERSHEY_DUPLEX, 0.3, BLACK, 1)
                highCount += 1

        cv2.rectangle(image_frame, (10, 10),(180, 60), GREY, cv2.FILLED)
        cv2.putText(image_frame, "--", (30,30), cv2.FONT_HERSHEY_DUPLEX, 0.5, RED, 1, cv2.LINE_AA)
        cv2.putText(image_frame, f'HIGH RISK: {str(highCount)} people', (60,30), cv2.FONT_HERSHEY_DUPLEX, 0.3, BLACK, 1)
        cv2.putText(image_frame, "--", (30,50), cv2.FONT_HERSHEY_DUPLEX, 0.5, YELLOW, 1, cv2.LINE_AA)
        cv2.putText(image_frame, f'LOW RISK : {str(lowCount)} people', (60,50), cv2.FONT_HERSHEY_DUPLEX, 0.3, BLACK, 1)
        
        cv2.imshow("Social Distance Detection Tracking", image_frame)
        count=count+1
        if cv2.waitKey(1) >= 0:  
            break 

    cap.release()

if __name__ == '__main__':
    
    '''
    mouse_pts = []    
    cv2.namedWindow("image")
    cv2.setMouseCallback("image", get_mouse_points)
    '''
    #320
    model, classes, colors, output_layers = loadYolo(320)
    start_time = time.time()
    social_distance_detection('yolo') 
    end_time = time.time()
    print('Time taken by model trained with YOLO320: {:.6f} seconds'.format(end_time - start_time))

    
    #416
    model, classes, colors, output_layers = loadYolo(416)
    start_time = time.time()
    social_distance_detection('yolo') 
    end_time = time.time()
    print('Time taken by model trained with YOLO416: {:.6f} seconds'.format(end_time - start_time))

    
    #608
    model, classes, colors, output_layers = loadYolo(608)
    start_time = time.time()
    social_distance_detection('yolo') 
    end_time = time.time()
    print('Time taken by model trained with YOLO608: {:.6f} seconds'.format(end_time - start_time))
    cv2.destroyAllWindows()

High speed, Less accuracy
Time taken by model trained with YOLO320: 135.574130 seconds
Moderate speed, Moderate accuracy
Time taken by model trained with YOLO416: 123.174186 seconds
Less speed, High accuracy
Time taken by model trained with YOLO608: 121.553725 seconds
